In [1]:
import pandas as pd
import numpy as np
from itertools import compress
from sklearn.metrics import cohen_kappa_score,f1_score,accuracy_score, precision_score, recall_score, classification_report, roc_auc_score, \
    hamming_loss


# [TODO] edit path to match experiments outputs
SOURCE_PATH_embeds = "/src_code/repo/experiments_results/user_aware_system/2020-10-02_12-09-30"
SOURCE_PATH_auds = "/src_code/repo/experiments_results/audio_system_single_label/2020-10-02_11-06-43"

In [2]:
# Read audio model's output and groundtruth
test_ground_truth = pd.read_csv("/src_code/repo/GroundTruth/test_set.csv")
test_groundtruth_from_model = np.loadtxt(SOURCE_PATH_auds + "/test_ground_truth_classes.txt",delimiter=',')
user_ids = np.loadtxt(SOURCE_PATH_auds + "/user_ids.txt",delimiter=',')
track_ids = np.loadtxt(SOURCE_PATH_auds + "/tracks_ids.txt",delimiter=',')
test_output = np.loadtxt(SOURCE_PATH_auds + "/predictions.out",delimiter=',')
test_output_one_hot = np.loadtxt(SOURCE_PATH_auds + "/test_output_one_hot.out",delimiter=',') ### WHAAAT

one_hot_a = test_ground_truth.copy()
one_hot_a.song_id = track_ids
one_hot_a.user_id = user_ids
one_hot_a.iloc[:,2:] = test_output_one_hot

preda = test_ground_truth.copy()
preda.song_id = track_ids
preda.user_id = user_ids
preda.iloc[:,2:] = test_output

In [3]:
# Read user's model's output and groundtruth
test_ground_truth = pd.read_csv("/src_code/repo/GroundTruth/test_set.csv")
test_groundtruth_from_model = np.loadtxt(SOURCE_PATH_embeds + "/test_ground_truth_classes.txt",delimiter=',')
user_ids = np.loadtxt(SOURCE_PATH_embeds + "/user_ids.txt",delimiter=',')
track_ids = np.loadtxt(SOURCE_PATH_embeds + "/tracks_ids.txt",delimiter=',')
test_output = np.loadtxt(SOURCE_PATH_embeds + "/predictions.out",delimiter=',')
max_idx = test_output.argmax(axis=1)
test_output_one_hot = np.zeros_like(test_output)
for counter in range(len(max_idx)):
    test_output_one_hot[counter,max_idx[counter]] = 1

# Format model output and groundtruth to a dataframe
gt = test_ground_truth.copy()
gt.song_id = track_ids
gt.user_id = user_ids
gt.iloc[:,2:] = test_groundtruth_from_model

one_hot_u = test_ground_truth.copy()
one_hot_u.song_id = track_ids
one_hot_u.user_id = user_ids
one_hot_u.iloc[:,2:] = test_output_one_hot

predu = test_ground_truth.copy()
predu.song_id = track_ids
predu.user_id = user_ids
predu.iloc[:,2:] = test_output

In [4]:
# create a frame of NAs 
LABELS_LIST = ['car', 'gym', 'happy', 'night', 'relax',
    'running', 'sad', 'summer', 'work', 'workout']
users_audio_f1 = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_f1 = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_audio_recall = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_recall = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_audio_acc = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_acc = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_audio_prec = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
users_embeds_prec = pd.DataFrame(gt.user_id.unique(),columns = ["user_id"])
for x in LABELS_LIST:
    users_audio_f1[x] = np.NAN
    users_embeds_f1[x] = np.NAN
    users_audio_recall[x] = np.NAN
    users_embeds_recall[x] = np.NAN
    users_audio_acc[x] = np.NAN
    users_embeds_acc[x] = np.NAN
    users_audio_prec[x] = np.NAN
    users_embeds_prec[x] = np.NAN
users_audio_f1 = users_audio_f1.set_index("user_id")
users_embeds_f1 = users_embeds_f1.set_index("user_id")
users_audio_recall = users_audio_recall.set_index("user_id")
users_embeds_recall = users_embeds_recall.set_index("user_id")
users_audio_acc = users_audio_acc.set_index("user_id")
users_embeds_acc = users_embeds_acc.set_index("user_id")
users_audio_prec = users_audio_prec.set_index("user_id")
users_embeds_prec = users_embeds_prec.set_index("user_id")

# compute scores for only present labels
for user in users_audio_f1.index:
    user_truth = gt[gt.user_id == user]
    user_preds_audio = one_hot_a.loc[preda.user_id == user]
    user_preds_embeds = one_hot_u.loc[predu.user_id == user]
    active_labels_idx = user_truth.sum() > 0
    active_labels_idx = active_labels_idx[2:]
    active_labels = list(compress(LABELS_LIST, active_labels_idx))
    for label in active_labels:
        users_audio_f1.loc[user][label] = f1_score(user_truth[label],user_preds_audio[label])
        users_embeds_f1.loc[user][label] = f1_score(user_truth[label],user_preds_embeds[label])
        users_audio_recall.loc[user][label] = recall_score(user_truth[label],user_preds_audio[label])
        users_embeds_recall.loc[user][label] = recall_score(user_truth[label],user_preds_embeds[label])
        users_audio_acc.loc[user][label] = accuracy_score(user_truth[label],user_preds_audio[label])
        users_embeds_acc.loc[user][label] = accuracy_score(user_truth[label],user_preds_embeds[label])
        users_audio_prec.loc[user][label] = precision_score(user_truth[label],user_preds_audio[label])
        users_embeds_prec.loc[user][label] = precision_score(user_truth[label],user_preds_embeds[label])
        
# averaging per user
user_audio_f1_mean = users_audio_f1.mean(axis=1)
user_embeds_f1_mean = users_embeds_f1.mean(axis=1)

users_audio_recall_mean = users_audio_recall.mean(axis=1)
user_embeds_recall_mean = users_embeds_recall.mean(axis=1)

user_audio_acc_mean = users_audio_acc.mean(axis=1)
user_embeds_acc_mean = users_embeds_acc.mean(axis=1)

user_audio_prec_mean = users_audio_prec.mean(axis=1)
user_embeds_prec_mean = users_embeds_prec.mean(axis=1)

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
results_df = pd.DataFrame(columns=['Audio', 'Audio+User'])
results_df.index.astype(str, copy=False)
results_df.loc[0] = [user_audio_acc_mean.mean(),user_embeds_acc_mean.mean()]
results_df.loc[1] = [users_audio_recall_mean.mean(),user_embeds_recall_mean.mean()]
results_df.loc[2] = [user_audio_prec_mean.mean(),user_embeds_prec_mean.mean()]
results_df.loc[3] = [user_audio_f1_mean.mean(),user_embeds_f1_mean.mean()]
results_df.index = ['Accuracy', 'Recall', 'Precision', 'f1-score']
results_df = results_df.T
results_df

,Accuracy,Recall,Precision,f1-score
Audio,0.215615,0.200012,0.308569,0.229246
Audio+User,0.245398,0.230948,0.354487,0.262005
